In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("IMDB Dataset.csv", on_bad_lines='skip')


In [4]:
df.shape

(50000, 2)

In [5]:
df = df.sample(10000, random_state=7) # // as we can see this contains 50k data but for working on small purpose lets take 10k data

In [6]:
df.head()

,review,sentiment
29430,I love most movies and I'm a big fan of Sean B...,negative
27750,A film that tends to get buried under prejudic...,positive
47782,Howard Brackett (Kevin Kline) is a teacher who...,positive
10498,"Like 'Singin' in the Rain', 'Cover Girl' has a...",negative
24747,Father of the Pride was the best new show to h...,positive


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 29430 to 42618
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [8]:
df.sentiment.value_counts() # this means using the df methoad we are counting the values of particular column ie how many positives and negatives ?
# insight is that this is binary classification positive and negative.!

sentiment
positive    5015
negative    4985
Name: count, dtype: int64

In [9]:
df['output'] = df['sentiment'].map({'positive': 1, 'negative': 0}) # here i am converting my sentiment labels from text to numbers so they can be used to train ml model
# positive will be 1 and negative will be 0

In [10]:
df['output'].value_counts() # again when we check the value count of output dataframe we can see that pos -> 1 & neg -> 0 and mapped correctly

output
1    5015
0    4985
Name: count, dtype: int64

In [11]:
df['length'] = df.review.apply(lambda x: len(x.split())) # in this we are just calcuating and adding len of the each review contains split will, spllit on the basis of
# white spaces and then it will form a list of each word like ["I","Love","This","Movie"] -> len 4 like this
# here i am doing feature engineering , this means basically small reviews may be not more detailed and vice versa...
# word count can be useful for the analyszing the patterns

In [12]:
df['length'].describe()

count    10000.000000
mean       230.031700
std        169.784644
min         10.000000
25%        126.000000
50%        172.000000
75%        278.000000
max       2125.000000
Name: length, dtype: float64

In [ ]:
# Text Preprocessing

In [13]:
!pip install nltk
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [14]:
# Download stopwords once
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
print(stop_words)

{'aren', 'down', "we're", 'doing', 'each', 'haven', "they've", 'them', 'why', 'we', 'themselves', 'yours', 'no', "they're", "they'd", 'both', 'if', "i'm", 'be', 'it', 'needn', 'against', 'as', "he'll", 'am', 'an', 'can', 'ma', 'a', 'their', 're', 'y', "you'll", 'hasn', 'by', 'has', 'nor', 'these', 'having', 'up', "we'll", 'some', 'when', "mustn't", 'but', 'from', "he's", 'being', 'any', 'all', "didn't", 'after', "hasn't", 'just', "needn't", 'does', 'not', 'whom', 'your', 'about', "wouldn't", "couldn't", 'most', 'below', 'was', 'll', 'did', "won't", 'yourself', "you're", 'herself', 's', 'hadn', 'are', 'here', 'because', "doesn't", 'again', 'hers', 'do', 'he', 'have', 'there', 'until', 'what', "hadn't", "should've", 'and', 'which', 'only', 'where', 'this', 'during', 't', 'into', "you'd", 'our', 'will', 'my', 'myself', 'she', "we'd", 'between', 'the', "he'd", 'had', "she'd", "it's", 'shouldn', 'out', "i'd", 'didn', "isn't", 'too', 'very', 'theirs', 've', 'couldn', 'how', 'in', 'its', "it'

In [16]:
# Remove not
stop_words.remove("not")

In [17]:
def expand_nt_contractions(text):
    # Replace n't with ' not'
    # Pattern explanation:
    # (\w+)n\'t → captures a word ending in n't like wasn't, didn't
    # \1 → inserts the first group (was, did, etc.)
    return re.sub(r"(\b\w+)n['’]t\b", r"\1 not", text)

In [18]:
sample = "I didn't like the movie."
expanded = expand_nt_contractions(sample)
print(expanded)

I did not like the movie.


In [19]:
def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 2. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 3. Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)

    # 4. Remove emojis and special characters (non-ASCII)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # 5. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 6. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 7. Expand nt Contractions
    text = expand_nt_contractions(text)

    # 8. Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Join cleaned words
    cleaned_text = " ".join(words)

    return cleaned_text


In [20]:
df['cleaned_review'] = df['review'].apply(preprocess_text)

In [21]:
df.sample(3)

,review,sentiment,output,length,cleaned_review
27219,"Well, maybe I'm just having a bad run with Hin...",negative,0,369,well maybe im bad run hindi movies lately aske...
38844,"""Revenge of the Zombies"" is a pretty weak and ...",negative,0,502,revenge zombies pretty weak barely passable zo...
7692,I saw this at the premiere in Melbourne<br /><...,negative,0,195,saw premiere melbourneit shallow twodimensiona...


In [22]:
df.iloc[31].review

"I have copy of this on VHS, I think they (The television networks) should play this every year for the next twenty years. So that we don't forget what was and that we remember not to do the same mistakes again. Like putting some people in the director's chair, where they don't belong. This movie Rappin' is like a vaudevillian musical, for those who can't sing, or act. This movie is as much fun as trying to teach the 'blind' to drive a city bus.<br /><br />John Hood, (Peebles) has just got out of prison and he's headed back to the old neighborhood. In serving time for an all-to-nice crime of necessity, of course. John heads back onto the old street and is greeted by kids dogs old ladies and his peer homeys as they dance and sing all along the way.<br /><br />I would recommend this if I was sentimental, or if in truth someone was smoking medicinal pot prescribed by a doctor for glaucoma. Either way this is a poorly directed, scripted, acted and even produced (I never thought I'd sat tha

In [23]:
df.iloc[31].cleaned_review

'copy vhs think television networks play every year next twenty years dont forget remember not mistakes like putting people directors chair dont belong movie rappin like vaudevillian musical cant sing act movie much fun trying teach blind drive city busjohn hood peebles got prison hes headed back old neighborhood serving time alltonice crime necessity course john heads back onto old street greeted kids dogs old ladies peer homeys dance sing along wayi would recommend sentimental truth someone smoking medicinal pot prescribed doctor glaucoma either way poorly directed scripted acted even produced never thought id sat satire ghetto life hood although think redeeming part story wannabe gang fight sequences dance numbers friends care neighbors want save ghetto torn cleaned forget sonny spoon mario could oscar comparison rap oh well find wanting laugh silly threequarters embarrassed sure drink first please watch responsibly stars better luck next time'

In [24]:
import nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

nltk.download('wordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')

nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
lemmatizer = WordNetLemmatizer()

In [26]:
def normalize_para(text):

    # Split words
    words = word_tokenize(text)

    # Get lemmas
    lemmas = [lemmatizer.lemmatize(word, pos="v") for word in words]

    # Join Lemmas words
    lemmatized = " ".join(lemmas)
    return lemmatized


In [27]:
df['normalized_review'] = df['cleaned_review'].apply(normalize_para)

In [28]:
df.iloc[31].normalized_review

'copy vhs think television network play every year next twenty years dont forget remember not mistake like put people directors chair dont belong movie rappin like vaudevillian musical cant sing act movie much fun try teach blind drive city busjohn hood peebles get prison hes head back old neighborhood serve time alltonice crime necessity course john head back onto old street greet kid dog old ladies peer homeys dance sing along wayi would recommend sentimental truth someone smoke medicinal pot prescribe doctor glaucoma either way poorly direct script act even produce never think id sit satire ghetto life hood although think redeem part story wannabe gang fight sequence dance number friends care neighbor want save ghetto tear clean forget sonny spoon mario could oscar comparison rap oh well find want laugh silly threequarters embarrass sure drink first please watch responsibly star better luck next time'

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
# 1. First split: train (70%) and temp (30%)
train_df, temp_df = train_test_split(df, train_size=0.7, random_state=42, shuffle=True)

In [31]:
# 2. Second split: temp into validation (10%) and test (20%)
# Since temp is 30%, to split it 1:2 ratio → validation = 1/3, test = 2/3
val_df, test_df = train_test_split(temp_df, test_size=2/3, random_state=42, shuffle=True)

In [32]:
# Check sizes
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Train: 7000, Val: 1000, Test: 2000


In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
vectorizer = CountVectorizer(min_df=3, max_df=0.95)
X_train = vectorizer.fit_transform(train_df['normalized_review'])

In [35]:
X_train.shape

(7000, 16174)

In [36]:
X_train[0]

<1x16174 sparse matrix of type '<class 'numpy.int64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [37]:
X_val = vectorizer.transform(val_df['normalized_review'])

In [38]:
X_val.shape

(1000, 16174)

In [39]:
X_test = vectorizer.transform(test_df['normalized_review'])

In [40]:
X_test.shape

(2000, 16174)

In [41]:
y_train = train_df["output"].values

In [42]:
y_val = val_df["output"].values

In [43]:
y_test = test_df["output"].values

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [45]:
# Define range of C values
c_values = [0.001, 0.01, 0.1, 1, 10, 100]
best_val_acc = 0
best_c = None
best_model = None

In [46]:
for c in c_values:
    model = LogisticRegression(C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)*100
    print(f"C={c} | Validation Accuracy = {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_c = c
        best_model = model


C=0.001 | Validation Accuracy = 83.40%
C=0.01 | Validation Accuracy = 85.60%
C=0.1 | Validation Accuracy = 87.00%
C=1 | Validation Accuracy = 86.00%
C=10 | Validation Accuracy = 85.20%
C=100 | Validation Accuracy = 85.10%


In [47]:
print(f"Best C = {best_c}")

Best C = 0.1


In [48]:
best_model

LogisticRegression(C=0.1, max_iter=1000, random_state=42)

In [49]:
y_test_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)*100
print(f"Test Accuracy = {test_acc:.2f}%")

Test Accuracy = 86.00%


In [50]:
import joblib

# Save the vectorizer and the best model
joblib.dump(vectorizer, 'bow_vectorizer.pkl')
joblib.dump(best_model, 'bow_classifier.pkl')

print("Models saved successfully!")
print(f"Vectorizer features: {len(vectorizer.get_feature_names_out())}")
print(f"Best model C value: {best_c}")

Models saved successfully!
Vectorizer features: 16174
Best model C value: 0.1
